In [1]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 780 kB/s 


Importing necessary libraries

In [15]:
from tensorflow.keras.layers import Dense,LSTM,Embedding,Dropout,Bidirectional,GRU,Flatten,BatchNormalization,Conv1D,MaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras import preprocessing
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
import pandas as pd 
import numpy as np
from sklearn.feature_extraction import FeatureHasher
from tensorflow.keras.metrics import MeanSquaredLogarithmicError
import category_encoders as ce
from sklearn.preprocessing import MinMaxScaler
import joblib

Importing the final datset after gaining insights from EDA and also performing a bit of feature engineering like what features impact the predictions more.

In [2]:
data = pd.read_csv('/content/drive/MyDrive/Merge3.csv',low_memory=False)
data.head()



,id,date,store_nbr,family,sales,onpromotion,type,locale,locale_name,description,oil_price,stores.store_nbr,stores.type,stores.cluster
0,3370,2013-01-02 00:00:00.000,53,BOOKS,0,0,NaN,NaN,NaN,NaN,93.14,53,D,13
1,5152,2013-01-03 00:00:00.000,53,BOOKS,0,0,NaN,NaN,NaN,NaN,92.97,53,D,13
2,6934,2013-01-04 00:00:00.000,53,BOOKS,0,0,NaN,NaN,NaN,NaN,93.12,53,D,13
3,10498,2013-01-06 00:00:00.000,53,BOOKS,0,0,NaN,NaN,NaN,NaN,93.20,53,D,13
4,12280,2013-01-07 00:00:00.000,53,BOOKS,0,0,NaN,NaN,NaN,NaN,93.20,53,D,13


Since the records are not sorted by date and as this is a forecasting problem the sequence or order of time stamps prove to be crucial in predictions , so sorting the data by date.

In [3]:
# data['date']=data['date'].apply(pd.to_datetime)
data = data.sort_values(by=['date'])


In [ ]:
data.head()

,id,date,store_nbr,family,sales,onpromotion,type,locale,locale_name,description,oil_price,stores.store_nbr,stores.type,stores.cluster
428261,1063,2013-01-01 00:00:00.000,39,CLEANING,0,0,Holiday,National,Ecuador,Primer dia del ano,93.14,39,B,6
428433,872,2013-01-01 00:00:00.000,33,HARDWARE,0,0,Holiday,National,Ecuador,Primer dia del ano,93.14,33,C,3
428432,871,2013-01-01 00:00:00.000,33,GROCERY II,0,0,Holiday,National,Ecuador,Primer dia del ano,93.14,33,C,3
428431,870,2013-01-01 00:00:00.000,33,GROCERY I,0,0,Holiday,National,Ecuador,Primer dia del ano,93.14,33,C,3
428430,869,2013-01-01 00:00:00.000,33,FROZEN FOODS,0,0,Holiday,National,Ecuador,Primer dia del ano,93.14,33,C,3


Filling null values to the NaN values in the data

In [4]:
data['description']=data['description'].fillna('null')
data['locale_name']=data['locale_name'].fillna('null')
data['locale']=data['locale'].fillna('null')
data['type'] = data['type'].fillna('null')

In [5]:
targets = data['sales'].copy()

data = data.drop(['stores.store_nbr','id','date','sales'], axis =1)

In [7]:
data.head()


,store_nbr,family,onpromotion,type,locale,locale_name,description,oil_price,stores.type,stores.cluster
428261,39,CLEANING,0,Holiday,National,Ecuador,Primer dia del ano,93.14,B,6
428433,33,HARDWARE,0,Holiday,National,Ecuador,Primer dia del ano,93.14,C,3
428432,33,GROCERY II,0,Holiday,National,Ecuador,Primer dia del ano,93.14,C,3
428431,33,GROCERY I,0,Holiday,National,Ecuador,Primer dia del ano,93.14,C,3
428430,33,FROZEN FOODS,0,Holiday,National,Ecuador,Primer dia del ano,93.14,C,3


From feature engineering and EDA it was understood that , the following categorical proved to be useful in predicting sales , but since the number of categorical is more , we try to encode each of them using binary encoding technique.

In [6]:
encoder = ce.BinaryEncoder(cols = ['type','family','locale','locale_name','description','stores.type'])
enc = encoder.fit(data)
data_encoded = encoder.transform(data)
encoder_file = '/content/drive/MyDrive/Colab Notebooks/Sales Forecasting/encoder.pkl'
joblib.dump(value=enc, 
            filename=encoder_file)

['/content/drive/MyDrive/Colab Notebooks/Sales Forecasting/encoder.pkl']

Dropping unnecessary columns 

In [ ]:

# data_encoded = data_encoded.drop(['stores.store_nbr','id','date'], axis =1)

In [ ]:
len(data_encoded)


3000888

Defining my own data generator , which generates outputs of past days sales data from all stores and also sales of the next day across all stores. Sampling wasnt performed as each timestamp pertains to a sale of a particluar product in a store . Since the data is in a orderly manner , it was a bit easier to generate a series of past and future sales. The number of days to look back is defined by the variable lookback and size_of_day corresponds to the total number of timestamps in a particluar day.

In [21]:
def my_gen(data,t,lookback,size_of_day):
  i=0
  sample=[]
  targets=[]
  while 1:
    seq1,seq2 = [],[]
    if i + (lookback+3)*size_of_day > data.shape[0]:
      return sample,targets
      break
    else:
      
      seq1 = data[i:i+(lookback*size_of_day)]
      seq2 = t[i+(lookback*size_of_day):i+(lookback*size_of_day)+(size_of_day)]
    sample.append(seq1)
    targets.append(seq2)
    i = i + size_of_day


In [ ]:
data_encoded.head()
# columns = data_encoded.column

428261    0
428433    0
428432    0
428431    0
428430    0
Name: sales, dtype: int64

Splitting the data into a 70 and 30 split and Normalizing the data using sklearn's MinMaxScaler and 

In [8]:
mx = MinMaxScaler()
train_data = data_encoded[:2094456]
t_targets = targets[:2094456]
test_data = data_encoded[2094457:]
te_targets = targets[2094457:]
train_q = mx.fit_transform(train_data)
test_q = mx.fit_transform(test_data)


In [23]:
print(targets)

428261       0
428433       0
428432       0
428431       0
428430       0
          ... 
2995139      0
2893257    187
2893256    431
2893254    380
3000887      0
Name: sales, Length: 3000888, dtype: int64


In [ ]:
print(train_q.shape)

(2094456, 32)


Getting our train input sequences and their targets , and test(validation )input sequences and their targets from our generator function

In [22]:
gen_sample, gen_targets = my_gen(train_q,t_targets,5,1781)
gen_sample = np.array(gen_sample)
gen_targtes = np.array(gen_targets)
test_sample, test_targets = my_gen(test_q,te_targets,5,1781)
test_sample = np.array(test_sample)
test_targets = np.array(test_targets)

In [11]:
gen_sample.shape

(1169, 8905, 31)

Building a Sequential Model using 4 Stacked Bidirectional LSTM layers and a model head comprising 2 Dense layers. Optimizer used is adam and loss function is mean absolute error and metric is MeanSquaredLogarithmicError as it was the metric used for evalution in the competition

In [30]:
model = Sequential()
model.add(Conv1D(32, 5, activation='relu',
input_shape=(8905, 31)))
model.add(MaxPooling1D(3))
model.add(Conv1D(32, 5, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(64, 5, activation='relu'))
model.add(Bidirectional( LSTM(32,return_sequences=True)))
model.add(Bidirectional( LSTM(32,return_sequences=True)))
model.add(Bidirectional( LSTM(32)))
model.add(Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(1781))
model.compile(optimizer='adam',loss='mae',metrics=[MeanSquaredLogarithmicError()])

Finally fitting the model

In [31]:
model_final = model.fit(x=gen_sample,y=gen_targtes,batch_size=8,epochs=10,validation_data=(test_sample,test_targets))

Epoch 1/10
147/147 [==============================] - 120s 726ms/step - loss: 312.8568 - mean_squared_logarithmic_error: 10.2140 - val_loss: 460.7656 - val_mean_squared_logarithmic_error: 9.7835
Epoch 2/10
147/147 [==============================] - 114s 778ms/step - loss: 312.3572 - mean_squared_logarithmic_error: 8.1115 - val_loss: 460.2662 - val_mean_squared_logarithmic_error: 9.1658
Epoch 3/10
147/147 [==============================] - 102s 700ms/step - loss: 312.2922 - mean_squared_logarithmic_error: 8.0204 - val_loss: 460.2872 - val_mean_squared_logarithmic_error: 9.2347
Epoch 4/10
147/147 [==============================] - 102s 696ms/step - loss: 312.2666 - mean_squared_logarithmic_error: 7.9938 - val_loss: 460.1179 - val_mean_squared_logarithmic_error: 9.0137
Epoch 5/10
147/147 [==============================] - 115s 790ms/step - loss: 312.2640 - mean_squared_logarithmic_error: 7.9429 - val_loss: 460.3941 - val_mean_squared_logarithmic_error: 9.4152
Epoch 6/10
147/147 [=========

In [ ]:
encoder_file = '/content/drive/MyDrive/Colab Notebooks/Sales Forecasting/encoder.pkl'
joblib.dump(value=[encoder], 
            filename=encoder_file)
model.save('/content/drive/MyDrive/Colab Notebooks/Sales Forecasting/TS_Model.hdf5')
